In [44]:
from sage.manifolds.operators import * # div, curl

# Vector field, $S^2$ 

cf. http://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/differentiable/vectorfield.html

In [11]:
S2 = Manifold(2, 'S^2')
U = S2.open_subset('U') # the open set covered by spherical coordinates
XS.<th, ph> = U.chart(r'th:(0,pi):\theta ph:(0,2*pi):\phi')
R3 = Manifold(3, 'R^3')
X3.<x, y, z> = R3.chart()
F = S2.diff_map(R3, {(XS, X3): [sin(th)*cos(ph), sin(th)*sin(ph), cos(th)]}, name='F')
F.display() # the standard embedding of S^2 into R^3

F: S^2 --> R^3
on U: (th, ph) |--> (x, y, z) = (cos(ph)*sin(th), sin(ph)*sin(th), cos(th))

In [12]:
v = XS.frame()[1]; v # the coordinate vector d/dphi

Vector field d/dph on the Open subset U of the 2-dimensional differentiable manifold S^2

In [14]:
graph_v = v.plot(chart=X3, mapping=F, label_axes=False)
graph_S2 = XS.plot(chart=X3, mapping=F, number_values=9)

In [15]:
graph_v + graph_S2

Graphics3d Object

# Tensor fields, on sphere $S^2$

In [18]:
S2 = Manifold(2, 'S^2') # the 2-dimensional sphere S^2
U = S2.open_subset('U') # complement of the North pole
c_xy.<x, y> = U.chart() # stereographic coordinates from the North pole
V = S2.open_subset('V') # complement of the South pole
c_uv.<u, v> = V.chart() # stereographic coordinates from the South pole
S2.declare_union(U, V) # S^2 is the union of U and V
xy_to_uv = \
    c_xy.transition_map(
        c_uv, (x / (x^2 + y^2), y / (x^2 + y^2)), 
        intersection_name='W', 
        restrictions1=x^2 + y^2 != 0,
        restrictions2=u^2 + v^2 !=0)
uv_to_xy = xy_to_uv.inverse()
W = U.intersection(V)

In [20]:
t = S2.tensor_field(0, 2, name='t'); t

Tensor field t of type (0,2) on the 2-dimensional differentiable manifold S^2

In [21]:
t.parent()

Module T^(0,2)(S^2) of type-(0,2) tensors fields on the 2-dimensional differentiable manifold S^2

In [22]:
t.parent().category()

Category of modules over Algebra of differentiable scalar fields on the 2-dimensional differentiable manifold S^2

The parent of `t` is not a free module, for the sphere $S^2$ is not parallelizable.

In [23]:
isinstance(t.parent(), FiniteRankFreeModule)

False

To fully define `t`, we have to specify its components in some vector frames defined on subsets of $S^2$; let us start by the open subset `U`:

In [24]:
eU = c_xy.frame()
t[eU, :] = [[1,0], [-2, 3]]
t.display(eU)

t = dx*dx - 2 dy*dx + 3 dy*dy

To set the components of `t` on `V` consistently, we copy the expressions of the components in the common subset `W`:

In [25]:
eV = c_uv.frame()
eVW = eV.restrict(W)
c_uvW = c_uv.restrict(W)

In [26]:
%%time
t[eV, 0, 0] = t[eVW, 0, 0, c_uvW].expr() # long time
t[eV, 0, 1] = t[eVW, 0, 1, c_uvW].expr() # long time
t[eV, 1, 0] = t[eVW, 1, 0, c_uvW].expr() # long time
t[eV, 1, 1] = t[eVW, 1, 1, c_uvW].expr() # long time

CPU times: user 1.28 s, sys: 16.4 ms, total: 1.3 s
Wall time: 1.12 s


Actually, the above operation can be performed in a single line by means of the method `add_comp_by_continuation()`

In [27]:
t.add_comp_by_continuation(eV, W, chart=c_uv) # long time

At this stage, `t` is fully defined, having components in frames `eU` and `eV` and the union of the domains of `eU` and `eV` being the whole manifold:

In [28]:
t.display(eV) # long time

t = (u^4 - 4*u^3*v + 10*u^2*v^2 + 4*u*v^3 + v^4)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) du*du - 4*(u^3*v + 2*u^2*v^2 - u*v^3)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) du*dv + 2*(u^4 - 2*u^3*v - 2*u^2*v^2 + 2*u*v^3 + v^4)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) dv*du + (3*u^4 + 4*u^3*v - 2*u^2*v^2 - 4*u*v^3 + 3*v^4)/(u^8 + 4*u^6*v^2 + 6*u^4*v^4 + 4*u^2*v^6 + v^8) dv*dv

# Connections, Affine Connections

cf. https://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/differentiable/affine_connection.html

Affine connection on a 3-dimensional manifold:

In [29]:
M = Manifold(3, 'M', start_index=1)
c_xyz.<x, y, z> = M.chart()
nab = M.affine_connection('nabla', r'\nabla'); nab

Affine connection nabla on the 3-dimensional differentiable manifold M

A just-created connection has no connection coefficients:

In [30]:
nab._coefficients

{}

The connection coefficients relative to the manifold's default frame [here ($\partial/\partial x, \partial/ \partial y, \partial/ \partial z$)], are created by providing the relevant indices inside square brackets:

In [31]:
nab[1, 1, 2], nab[3, 2, 3] = x^2, y*z # Gamma^1_{12} = x^2, Gamma^3_{23} = yz
nab._coefficients

{Coordinate frame (M, (d/dx,d/dy,d/dz)): 3-indices components w.r.t. Coordinate frame (M, (d/dx,d/dy,d/dz))}

If not the default one, the vector frame w.r.t which the connection coefficients are defined can be specified as the first argument inside the square brackets; hence the above definition is equivalent to:

In [32]:
nab[c_xyz.frame(), 1, 1, 2], nab[c_xyz.frame(), 3, 2, 3] = x^2, y*z
nab._coefficients

{Coordinate frame (M, (d/dx,d/dy,d/dz)): 3-indices components w.r.t. Coordinate frame (M, (d/dx,d/dy,d/dz))}

Unset components are initialized to zero:

In [33]:
nab[:]

[[[0, x^2, 0], [0, 0, 0], [0, 0, 0]],
 [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
 [[0, 0, 0], [0, 0, y*z], [0, 0, 0]]]

# Euclidean Spaces 

cf. http://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/differentiable/euclidean.html

In [36]:
E.<x,y,z> = EuclideanSpace(3); E

Euclidean space E^3

In [37]:
latex(E)

\mathbb{E}^{3}

## Vector calculus in the Euclidean 3-space

A simple vector field on `E`

In [38]:
v = E.vector_field(-y, x, 0, name='v')
v.display()

v = -y e_x + x e_y

In [39]:
v[:]

[-y, x, 0]

The Euclidean norm of `v`:

In [40]:
s = norm(v); s

Scalar field |v| on the Euclidean space E^3

In [41]:
s.display()

|v|: E^3 --> R
   (x, y, z) |--> sqrt(x^2 + y^2)

In [42]:
s.expr()

sqrt(x^2 + y^2)

The divergence of `v` is zero:

In [45]:
div(v)

Scalar field div(v) on the Euclidean space E^3

In [46]:
div(v).display()

div(v): E^3 --> R
   (x, y, z) |--> 0

while its curl is a constant vector field along `e_z`:

In [47]:
w = curl(v); w

Vector field curl(v) on the Euclidean space E^3

In [48]:
w.display()

curl(v) = 2 e_z

The gradient of a scalar field:

## Coordinates

It is endowed with a default coordinate chart, which is that of Cartesian coordinates $(x, y, z)$:

In [49]:
E.atlas()

[Chart (E^3, (x, y, z))]

In [50]:
E.default_chart()

Chart (E^3, (x, y, z))

In [52]:
cartesian = E.cartesian_coordinates()
cartesian is E.default_chart()

True

`E` is endowed with a default metric tensor, which defines the Euclidean scalar product:

In [53]:
g = E.metric(); g

Riemannian metric g on the Euclidean space E^3

In [54]:
g.display()

g = dx*dx + dy*dy + dz*dz

### Cylindrical coordinates, `cylindrical_coordinates`

In [56]:
E.cylindrical_coordinates()

Chart (E^3, (rh, ph, z))

The coordinate variables are returned by the square bracket operator:

In [57]:
print(E.cylindrical_coordinates()[1])
print(E.cylindrical_coordinates()[3])
print(E.cylindrical_coordinates()[:])

rh
z
(rh, ph, z)


They can also be obtained via the operator `<,>`:

In [58]:
cylindrical.<rh,ph,z> = E.cylindrical_coordinates()
cylindrical

Chart (E^3, (rh, ph, z))

### Spherical_coordinates, `spherical_coordinates`

In [59]:
E.spherical_coordinates()

Chart (E^3, (r, th, ph))

In [60]:
E.spherical_coordinates().coord_range()

r: (0, +oo); th: (0, pi); ph: [0, 2*pi] (periodic)

The relation to Cartesian coordinates is:

In [61]:
E.coord_change(E.spherical_coordinates(), E.cartesian_coordinates()).display()

x = r*cos(ph)*sin(th)
y = r*sin(ph)*sin(th)
z = r*cos(th)

In [62]:
E.coord_change(E.cartesian_coordinates(), E.spherical_coordinates()).display()

r = sqrt(x^2 + y^2 + z^2)
th = arctan2(sqrt(x^2 + y^2), z)
ph = arctan2(y, x)

#### `spherical_frame()`

In [63]:
E.spherical_frame()

Vector frame (E^3, (e_r,e_th,e_ph))

In [64]:
E.spherical_frame()[1]

Vector field e_r on the Euclidean space E^3

In [65]:
E.spherical_frame()[:]

(Vector field e_r on the Euclidean space E^3,
 Vector field e_th on the Euclidean space E^3,
 Vector field e_ph on the Euclidean space E^3)